## Object Tracking & Counting YOLOv8

In [1]:
###############
# PIP Install #
###############

!pip install yolo
!pip install ultralytics


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
###########
# Imports #
###########

import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

In [2]:
######################
# Parameters & model #
######################

## Initialize IDs sets ##
car_ids = set()
truck_ids = set()

## Dictionary to track the last known Y position of each ID ##
last_positions = {}

car_in, car_out, truck_in, truck_out = 0, 0, 0, 0

## Initialize segmentation model ##
model = YOLO("yolov8l-seg.pt")
filename = "video001"

## Create VideoCapture object ##
cap = cv2.VideoCapture(f"/home/valerio/code/ValerioCann/Modèles/Data/Videos//{filename}.mp4")

## Retrieve height, widht & frame per second ##
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## Create VideoWriter object ##
out = cv2.VideoWriter(f'{filename}.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))


##########################
# Tracking & Annotations #
##########################

# Sets a horizontal line in the middle of the image for counting
line_y = h // 2
        
while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break
        
    ## instantiates annotator ##
    annotator = Annotator(im0, line_width=2)
    
    ## Track objects ##
    results = model.track(im0, persist=True)

    ## Retrieve masks & track ids ##
    if results[0].boxes.id is not None and results[0].masks is not None:
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().cpu().tolist()

        
        ## Draws segmentation boxes on detected objects ##
        for mask, track_id in zip(masks, track_ids):
            ## Calculation of the central position of the vehicle (we will use the center of the mask) ##
            center_y = int(np.mean(np.array(mask)[:, 1]))  # Average of mask Y coordinates

            # Checks if the vehicle has already been tracked
            if track_id in last_positions:
                # Determines direction based on previous position
                if last_positions[track_id] < line_y <= center_y:
                    # Vehicle moves down - IN
                    if track_id in car_ids: car_in += 1
                    else: truck_in += 1
                elif last_positions[track_id] > line_y >= center_y:
                    # Vehicle moves up - OUT
                    if track_id in car_ids: car_out += 1
                    else: truck_out += 1

            # Updating the vehicle's last known position
            last_positions[track_id] = center_y
            annotator.seg_bbox(mask=mask,
                               mask_color=colors(track_id, True),
                               track_label=str(track_id))

            # Keep track of unique car and truck IDs
            if track_id not in car_ids and track_id not in truck_ids:
                area = cv2.contourArea(mask)
                if area > 50000:  # Adjust threshold for distinguishing between cars and trucks
                    truck_ids.add(track_id)
                else:
                    car_ids.add(track_id)

    
###################
# Text parameters #
###################


    ## Font size for text ##
    fontScale = 2.0  
    
    ## Space around text in rectangle ##
    padding = 10  
    
    line_spacing = 20
    
    ## Text thickness ##
    thickness = 3
    
    ## Text ##
    text_cars = f"Cars IN: {car_in}, Cars OUT: {car_out}"
    text_trucks = f"Trucks IN: {truck_in}, Trucks OUT: {truck_out}"
    
    ## Get text size ##
    (text_width_cars, text_height_cars), _ = cv2.getTextSize(text_cars, cv2.FONT_HERSHEY_SIMPLEX, fontScale, 2)
    (text_width_trucks, text_height_trucks), _ = cv2.getTextSize(text_trucks, cv2.FONT_HERSHEY_SIMPLEX, fontScale, 2)

    ## Calculating the total width and height needed for the rectangle ##
    total_text_width = max(text_width_cars, text_width_trucks)
    total_text_height = text_height_cars + text_height_trucks + padding + line_spacing  # Ajoutez un peu d'espace (padding) entre les lignes
    
    ## Coordinates of the rectangle around the text ##
    rectangle_top_left = (0, 0)  # En haut à gauche de l'image
    rectangle_bottom_right = (total_text_width + 2 * padding, total_text_height + 3 * padding + 10)

    ## Rectangle color - BGR ##
    rectangle_color = (0, 0, 255) 

    ## Draw the rectangle ##
    cv2.rectangle(im0, rectangle_top_left, rectangle_bottom_right, rectangle_color, cv2.FILLED)
    
    ## Text color - BGR ##
    text_color = (255, 255, 255)
    
    ## Add text ##
    cv2.putText(im0, text_cars, (padding, text_height_cars + padding), cv2.FONT_HERSHEY_SIMPLEX, fontScale, text_color, 2, cv2.LINE_AA)
    cv2.putText(im0, text_trucks, (padding, 2 * text_height_cars + 2 * padding + line_spacing), cv2.FONT_HERSHEY_SIMPLEX, fontScale, text_color, 2, cv2.LINE_AA)

    
#############################################
# Output video creation, saving and display #
#############################################


    ## Draw the counting line on the image ##
    cv2.line(im0, (0, line_y), (w, line_y), (0, 0, 255), 3)  # Ligne jaune

    ## Adds a new frame to the output video ##
    out.write(im0)
    
    ## displays the frame ##
    cv2.imshow("instance-segmentation-object-tracking", im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

  0%|          | 0.00/88.1M [00:00<?, ?B/s]


0: 384x640 8 cars, 5 trucks, 797.6ms
Speed: 5.6ms preprocess, 797.6ms inference, 19.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 5 trucks, 793.9ms
Speed: 4.3ms preprocess, 793.9ms inference, 14.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 5 trucks, 674.7ms
Speed: 1.9ms preprocess, 674.7ms inference, 8.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 5 trucks, 654.8ms
Speed: 1.9ms preprocess, 654.8ms inference, 12.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 5 trucks, 1 stop sign, 675.1ms
Speed: 2.6ms preprocess, 675.1ms inference, 16.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 5 trucks, 1 stop sign, 553.4ms
Speed: 2.0ms preprocess, 553.4ms inference, 12.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 5 trucks, 1 stop sign, 627.7ms
Speed: 4.6ms preprocess, 627.7ms inference, 12.5ms postprocess per image at shape (1, 3, 384, 640)

0:


0: 384x640 1 person, 7 cars, 7 trucks, 616.1ms
Speed: 4.6ms preprocess, 616.1ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 7 trucks, 578.3ms
Speed: 2.2ms preprocess, 578.3ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 7 trucks, 644.2ms
Speed: 2.6ms preprocess, 644.2ms inference, 9.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 7 trucks, 602.3ms
Speed: 3.7ms preprocess, 602.3ms inference, 9.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 7 trucks, 646.1ms
Speed: 2.3ms preprocess, 646.1ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 7 trucks, 597.6ms
Speed: 2.9ms preprocess, 597.6ms inference, 9.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 7 trucks, 608.5ms
Speed: 2.5ms preprocess, 608.5ms inference, 7.0ms postprocess per image at s


0: 384x640 6 cars, 9 trucks, 617.1ms
Speed: 2.6ms preprocess, 617.1ms inference, 9.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 9 trucks, 546.0ms
Speed: 3.5ms preprocess, 546.0ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 7 trucks, 631.2ms
Speed: 4.7ms preprocess, 631.2ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 8 trucks, 589.0ms
Speed: 5.2ms preprocess, 589.0ms inference, 7.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 8 trucks, 583.5ms
Speed: 3.1ms preprocess, 583.5ms inference, 5.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 8 trucks, 590.5ms
Speed: 2.3ms preprocess, 590.5ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 8 trucks, 566.7ms
Speed: 2.6ms preprocess, 566.7ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 8 trucks, 667.9ms
Speed: 3.2


0: 384x640 8 cars, 7 trucks, 555.7ms
Speed: 4.5ms preprocess, 555.7ms inference, 8.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 6 trucks, 612.3ms
Speed: 2.7ms preprocess, 612.3ms inference, 8.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 7 trucks, 665.2ms
Speed: 4.5ms preprocess, 665.2ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 7 trucks, 661.7ms
Speed: 2.4ms preprocess, 661.7ms inference, 11.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 6 trucks, 688.8ms
Speed: 3.3ms preprocess, 688.8ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 7 trucks, 1 stop sign, 709.2ms
Speed: 5.2ms preprocess, 709.2ms inference, 9.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 5 trucks, 1 stop sign, 698.0ms
Speed: 2.5ms preprocess, 698.0ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 


0: 384x640 9 cars, 7 trucks, 1 stop sign, 683.2ms
Speed: 3.6ms preprocess, 683.2ms inference, 11.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 7 trucks, 1 stop sign, 664.3ms
Speed: 2.2ms preprocess, 664.3ms inference, 9.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 7 trucks, 1 stop sign, 613.9ms
Speed: 4.7ms preprocess, 613.9ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 7 trucks, 1 stop sign, 562.5ms
Speed: 2.5ms preprocess, 562.5ms inference, 8.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 7 trucks, 1 stop sign, 626.5ms
Speed: 2.7ms preprocess, 626.5ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 6 trucks, 1 stop sign, 679.0ms
Speed: 2.4ms preprocess, 679.0ms inference, 9.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 7 trucks, 1 stop sign, 529.5ms
Speed: 2.3ms preprocess, 529.5ms inference, 6.3ms post


0: 384x640 7 cars, 6 trucks, 660.3ms
Speed: 2.7ms preprocess, 660.3ms inference, 8.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 6 trucks, 642.6ms
Speed: 5.0ms preprocess, 642.6ms inference, 8.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 6 trucks, 618.8ms
Speed: 2.6ms preprocess, 618.8ms inference, 6.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 6 trucks, 573.4ms
Speed: 2.9ms preprocess, 573.4ms inference, 8.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 7 trucks, 625.5ms
Speed: 1.9ms preprocess, 625.5ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 7 trucks, 1 stop sign, 662.2ms
Speed: 4.5ms preprocess, 662.2ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 7 trucks, 1 stop sign, 608.5ms
Speed: 4.8ms preprocess, 608.5ms inference, 8.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 7 